In [13]:
# ! pip install pandas
# ! pip install requests
# ! pip install plotly
# ! pip install datetime
# ! pip install os
# ! pip freeze = requirements.txt

In [14]:
import pandas as pd
import requests as r
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime, timedelta
import numpy as np
import os
import time

In [15]:
pwd = os.getcwd()
if 'L2 TVL' in pwd:
    prepend = ''
else:
    prepend = 'L2 TVL/'

In [16]:

api_str = 'https://api.llama.fi/protocol/'

# Protocol Incentive Start Dates
# NOTE: This should be when the in-app incentives began, not any external incentives (i.e. DEX pools)
protocols = [
    # name, incentive start date
        #  ['velodrome',          '2022-07-13']
        # ,['pooltogether',       '2022-07-14']
        # ,['lyra',               '2022-08-02']
        # ,['rubicon',            '2022-07-15']
        # ,['perpetual-protocol', '2022-07-14']
        # ,['thales',             '2022-07-14'] #TVL not relevant
        ['aave-v3',            '2022-01-01']
        # ,['wepiggy',            '2022-08-03']
        # ,['stargate',           '2022-08-05']
        # ,['pika-protocol',      '2022-08-29']
        # ,['synthetix',          '2022-08-25'] #This is when Curve incentives started, so not really 1:1
        # ,['pickle',             '2022-09-09']
        # ,['aelin',              '2022-09-12']
        # ,['polynomial-protocol','2022-09-14']
        # ,['xtoken',             '2022-09-19']
        # ,['hop-protocol',       '2022-09-22']
        # ,['beethoven-x',        '2022-09-29']
        # ,['uniswap-v3',         '2022-10-26']
        # ,['arrakis-finance',    '2022-10-26']
        # ,['gamma',              '2022-10-26']
        ]
# print(protocols[0])
prod = []
s = r.Session()

for prot in protocols:
    print(api_str + prot[0])
    try:
        tp = s.get(api_str + prot[0]).json()['chainTvls']['Avalanche']
        ad = pd.json_normalize( tp['tokens'] )
        ad_usd = pd.json_normalize( tp['tokensInUsd'] )
        if not ad.empty:
            ad = pd.melt(ad,id_vars = ['date'])
            ad = ad.rename(columns={'variable':'token','value':'token_value'})
            ad_usd = pd.melt(ad_usd,id_vars = ['date'])
            ad_usd = ad_usd.rename(columns={'variable':'token','value':'usd_value'})
            ad = ad.merge(ad_usd,on=['date','token'])
            
            ad['date'] = pd.to_datetime(ad['date'], unit ='s') #convert to days

            ad['token'] = ad['token'].str.replace('tokens.','', regex=False)
            ad['protocol'] = prot[0]
            ad['start_date'] = pd.to_datetime(prot[1])
            # ad['date'] = ad['date'] - timedelta(days=1) #change to eod vs sod
            prod.append(ad)
            time.sleep(0.5)
    except:
        continue

df_df = pd.concat(prod)

https://api.llama.fi/protocol/aave-v3


In [26]:
# df_df
df_df = df_df.fillna(0)
display(df_df)
# display(df_df)
# for prot in protocols:
#         print( prot[0] )

,date,token,token_value,usd_value,protocol,start_date
0,2022-03-14 00:00:00,DAI,1.00000,1.001000e+00,aave-v3,2022-01-01
1,2022-03-15 00:00:00,DAI,1.00000,9.988600e-01,aave-v3,2022-01-01
2,2022-03-16 00:00:00,DAI,1.00000,9.978300e-01,aave-v3,2022-01-01
3,2022-03-17 00:00:00,DAI,3.00000,2.999670e+00,aave-v3,2022-01-01
4,2022-03-18 00:00:00,DAI,10722.61585,1.072262e+04,aave-v3,2022-01-01
...,...,...,...,...,...,...
4211,2022-11-12 00:00:00,BTC.B,1969.58570,3.319256e+07,aave-v3,2022-01-01
4212,2022-11-13 00:00:00,BTC.B,2011.92925,3.365610e+07,aave-v3,2022-01-01
4213,2022-11-14 00:00:00,BTC.B,1980.18380,3.207025e+07,aave-v3,2022-01-01
4214,2022-11-15 00:00:00,BTC.B,1333.06833,2.217900e+07,aave-v3,2022-01-01


In [18]:
data_df = df_df.copy()#merge(cg_df, on=['date','token'],how='inner')

data_df = data_df[data_df['token_value'] > 0]

data_df.sort_values(by='date',inplace=True)
data_df['token_value'] = data_df['token_value'].replace(0, np.nan)
data_df['price_usd'] = data_df['usd_value']/data_df['token_value']

data_df['rank_desc'] = data_df.groupby(['protocol', 'token'])['date'].\
                            rank(method='dense',ascending=False).astype(int)

data_df.sort_values(by='date',inplace=True)

last_df = data_df[data_df['rank_desc'] == 1]
last_df = last_df.rename(columns={'price_usd':'last_price_usd'})
last_df = last_df[['token','protocol','last_price_usd']]
# display(last_df)

In [19]:
data_df = data_df.merge(last_df, on=['token','protocol'], how='left')

data_df['last_token_value'] = data_df.groupby(['token','protocol'])['token_value'].shift(1)
data_df['last_price_usd'] = data_df.groupby(['token','protocol'])['price_usd'].shift(1)
data_df['last_token_value'] = data_df['last_token_value'].fillna(0)

data_df['net_token_flow'] = data_df['token_value'] - data_df['last_token_value']
data_df['net_price_change'] = data_df['price_usd'] - data_df['last_price_usd']

data_df['net_dollar_flow'] = data_df['net_token_flow'] * data_df['price_usd']
data_df['last_price_flow'] = data_df['net_token_flow'] * data_df['last_price_usd']

data_df['net_price_stock_change'] = data_df['last_token_value'] * data_df['net_price_change']


# display(data_df)

In [20]:
# data_df[data_df['protocol']=='perpetual-protocol'].sort_values(by='date')
data_df.tail()
# data_df[(data_df['protocol'] == 'pooltogether') & (data_df['date'] >= '2022-10-06') & (data_df['date'] <= '2022-10-12')].tail(10)

,date,token,token_value,usd_value,protocol,start_date,price_usd,rank_desc,last_price_usd,last_token_value,net_token_flow,net_price_change,net_dollar_flow,last_price_flow,net_price_stock_change
2074,2022-11-15 22:59:35,avalanche-2,1.517926e+06,2.049200e+07,aave-v3,2022-01-01,13.500000,1,13.070000,1.523649e+06,-5723.15971,0.430000,-77262.656085,-74801.697410,655169.156705
2075,2022-11-15 22:59:35,FRAX,3.245648e+05,3.245106e+05,aave-v3,2022-01-01,0.999833,1,0.997232,3.257289e+05,-1164.09885,0.002601,-1163.904445,-1160.876624,847.220915
2076,2022-11-15 22:59:35,LINK,2.615901e+05,1.676793e+06,aave-v3,2022-01-01,6.410000,1,6.320000,2.241202e+05,37469.95566,0.090000,240182.415779,236810.119769,20170.815701
2077,2022-11-15 22:59:35,SAVAX,8.863347e+05,1.239096e+07,aave-v3,2022-01-01,13.980000,1,13.550000,8.613609e+05,24973.86101,0.430000,349134.576922,338395.816684,370385.177648
2078,2022-11-15 22:59:35,BTC.B,1.366182e+03,2.304511e+07,aave-v3,2022-01-01,16868.249959,1,16637.559990,1.333068e+03,33.11404,230.689969,558575.903860,550936.827003,307525.491550


In [21]:
netdf_df = data_df[data_df['date']>= data_df['start_date']][['date','protocol','net_dollar_flow','net_price_stock_change','last_price_flow','usd_value']]


netdf_df = netdf_df.groupby(['date','protocol']).sum(['net_dollar_flow','net_price_stock_change','last_price_flow','usd_value'])


netdf_df['tvl_change'] = netdf_df['usd_value'] - netdf_df.groupby(['protocol'])['usd_value'].shift(1)
netdf_df['error'] = netdf_df['tvl_change'] - (netdf_df['net_dollar_flow'] + netdf_df['net_price_stock_change'])


netdf_df['cumul_net_dollar_flow'] = netdf_df['net_dollar_flow'].groupby(['protocol']).cumsum()
netdf_df['cumul_last_price_net_dollar_flow'] = netdf_df['last_price_flow'].groupby(['protocol']).cumsum()
netdf_df['cumul_net_price_stock_change'] = netdf_df['net_price_stock_change'].groupby(['protocol']).cumsum()
netdf_df.reset_index(inplace=True)


In [22]:
netdf_df[(netdf_df['date'] >= '2022-10-06') & (netdf_df['date'] <= '2022-10-12')].tail(10)

,date,protocol,net_dollar_flow,net_price_stock_change,last_price_flow,usd_value,tvl_change,error,cumul_net_dollar_flow,cumul_last_price_net_dollar_flow,cumul_net_price_stock_change
206,2022-10-06,aave-v3,-1.269088e+07,8.067335e+05,-1.272697e+07,6.422966e+08,-1.188415e+07,7.823110e-08,1.518087e+09,1.048173e+09,-6.380618e+08
207,2022-10-07,aave-v3,-9.874048e+06,-1.200617e+06,-9.895215e+06,6.312220e+08,-1.107466e+07,5.587935e-08,1.508213e+09,1.038278e+09,-6.392624e+08
208,2022-10-08,aave-v3,3.857679e+05,-3.174785e+06,4.320806e+05,6.284330e+08,-2.789018e+06,-1.336448e-07,1.508599e+09,1.038710e+09,-6.424372e+08
209,2022-10-09,aave-v3,1.899556e+06,-1.554213e+06,1.896401e+06,6.287783e+08,3.453437e+05,1.082662e-07,1.510499e+09,1.040607e+09,-6.439914e+08
210,2022-10-10,aave-v3,-8.934544e+05,-5.491511e+04,-8.939770e+05,6.278299e+08,-9.483695e+05,-6.821938e-08,1.509605e+09,1.039713e+09,-6.440463e+08
211,2022-10-11,aave-v3,1.716499e+05,-3.839944e+06,2.236054e+05,6.241616e+08,-3.668294e+06,-9.313226e-09,1.509777e+09,1.039936e+09,-6.478862e+08
212,2022-10-12,aave-v3,4.551290e+06,4.550434e+05,4.521049e+06,6.291680e+08,5.006333e+06,9.126961e-08,1.514328e+09,1.044457e+09,-6.474312e+08


In [23]:
fig = px.line(netdf_df, x="date", y="net_dollar_flow", color="protocol", \
             title="Daily Net Dollar Flow since Program Announcement",\
            labels={
                     "date": "Day",
                     "net_dollar_flow": "Net Dollar Flow (N$F)"
                 }
            )
fig.update_layout(
    legend_title="App Name"
)
fig.update_layout(yaxis_tickprefix = '$')
fig.write_image(prepend + "img_outputs/svg/daily_ndf.svg")
fig.write_image(prepend + "img_outputs/png/daily_ndf.png")
fig.write_html(prepend + "img_outputs/daily_ndf.html", include_plotlyjs='cdn')

# cumul_fig = px.area(netdf_df, x="date", y="cumul_net_dollar_flow", color="protocol", \
#              title="Cumulative Dollar Flow since Program Announcement",\
#                    labels={
#                      "date": "Day",
#                      "cumul_net_dollar_flow": "Cumulative Net Dollar Flow (N$F)"
#                  }
#             ,areamode='group')
# cumul_fig.update_layout(yaxis_tickprefix = '$')
# cumul_fig.show()


cumul_fig = go.Figure()
proto_names = netdf_df['protocol'].drop_duplicates()
print(proto_names)
for p in proto_names:
    cumul_fig.add_trace(go.Scatter(x=netdf_df[netdf_df['protocol'] == p]['date'] \
                                   , y=netdf_df[netdf_df['protocol'] == p]['cumul_net_dollar_flow'] \
                                    ,name = p\
                                  ,fill='tozeroy')) # fill down to xaxis

cumul_fig.update_layout(yaxis_tickprefix = '$')
cumul_fig.update_layout(
    title="Cumulative Net Dollar Flow since Program Announcement",
    xaxis_title="Day",
    yaxis_title="Cumulative Net Dollar Flow (N$F)",
    legend_title="App Name",
#     color_discrete_map=px.colors.qualitative.G10
)
# cumul_fig.write_image(prepend + "img_outputs/svg/cumul_ndf.svg") #prepend + 
# cumul_fig.write_image(prepend + "img_outputs/png/cumul_ndf.png") #prepend + 
# cumul_fig.write_html(prepend + "img_outputs/cumul_ndf.html", include_plotlyjs='cdn')
cumul_fig.show()

fig_last = go.Figure()
proto_names = netdf_df['protocol'].drop_duplicates()
# print(proto_names)
for p in proto_names:
    fig_last.add_trace(go.Scatter(x=netdf_df[netdf_df['protocol'] == p]['date'] \
                                   , y=netdf_df[netdf_df['protocol'] == p]['cumul_last_price_net_dollar_flow'] \
                                    ,name = p\
                                  ,fill='tozeroy')) # fill down to xaxis

fig_last.update_layout(yaxis_tickprefix = '$')
fig_last.update_layout(
    title="Cumulative Net Dollar Flow since Program Announcement (At Most Recent Token Price)",
    xaxis_title="Day",
    yaxis_title="Cumulative Net Dollar Flow (N$F) - At Most Recent Price",
    legend_title="App Name",
#     color_discrete_map=px.colors.qualitative.G10
)
# fig_last.write_image(prepend + "img_outputs/svg/cumul_ndf_last_price.svg")
# fig_last.write_image(prepend + "img_outputs/png/cumul_ndf_last_price.png")
# fig_last.write_html(prepend + "img_outputs/cumul_ndf_last_price.html", include_plotlyjs='cdn')
# cumul_fig.show()


0    aave-v3
Name: protocol, dtype: object


In [24]:
# fig.show()
# cumul_fig.show()
print("yay")

yay


In [25]:
# ! jupyter nbconvert --to python optimism_app_net_flows.ipynb